In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import io
import joblib

# --- 1. Load Data ---
# In a real scenario, you would load from a file.
# For this example, we'll use the provided CSV data directly.
csv_data = """Food_Item,Source_Type,Storage_Type,Date_of_Collection,Color,Smell,Texture,Foreign_Particles,Appearance,Float_or_Settle_Test,Dissolves_in_Water,Foam_After_Mixing,Burn_Test_Result,Taste,Is_Adulterated,Suspected_Adulterant,Health_Risk_Level
Milk,Local Market,Container,2023-01-15,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Farmer,Sack,2023-01-16,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Turmeric,Vendor,Loose,2023-01-17,Too Bright,Chemical,Powdery,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Wheat,Farmer,Sack,2023-01-18,Normal,Normal,Dry,Stones,Dusty,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Chilli Powder,Local Market,Loose,2023-01-19,Too Bright,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Leaves Black Residue,Sharp,Yes,Color,Moderate
Tomato,Self-grown,Open,2023-01-20,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Milk,Vendor,Container,2023-01-21,Yellowish,Chemical,Sticky,None,Layered,Mixed,Forms Layer,Excessive Foam,Melts,Bitter,Yes,Detergent,Toxic
Rice,Local Market,Sack,2023-01-22,Faded,Rotten,Wet,Insects,Clumpy,Floats,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Urea,Moderate
Wheat,Farmer,Sack,2023-01-23,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Turmeric,Local Market,Covered,2023-01-24,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Milk,Farmer,Container,2023-01-25,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Chilli Powder,Vendor,Loose,2023-01-26,Normal,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,No,None,Safe
Rice,Farmer,Sack,2023-01-27,Normal,Normal,Dry,Husks,Dusty,Mixed,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Milk,Local Market,Container,2023-01-28,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Tomato,Vendor,Open,2023-01-29,Mixed Shades,Rotten,Sticky,None,Clumpy,Sinks,Leaves Residue,No Foam,Burns Cleanly,Not Tested,Yes,Color,Moderate
Turmeric,Farmer,Covered,2023-01-30,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Wheat,Local Market,Sack,2023-01-31,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Vendor,Container,2023-02-01,Yellowish,Chemical,Smooth,None,Layered,Mixed,Forms Layer,Excessive Foam,Plastic Smell,Bitter,Yes,Detergent,Toxic
Rice,Self-grown,Container,2023-02-02,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Chilli Powder,Local Market,Loose,2023-02-03,Normal,Strong,Powdery,Seeds,Dusty,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,Yes,Stone,Low Risk
Milk,Farmer,Container,2023-02-04,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Turmeric,Vendor,Loose,2023-02-05,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Wheat,Farmer,Sack,2023-02-06,Normal,No Smell,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Local Market,Container,2023-02-07,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Rice,Vendor,Sack,2023-02-08,Faded,Rotten,Wet,Insects,Clumpy,Floats,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Urea,Moderate
Tomato,Farmer,Open,2023-02-09,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Chilli Powder,Self-grown,Container,2023-02-10,Normal,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,No,None,Safe
Milk,Vendor,Container,2023-02-11,Yellowish,Chemical,Sticky,None,Layered,Mixed,Forms Layer,Excessive Foam,Melts,Bitter,Yes,Detergent,Toxic
Turmeric,Local Market,Covered,2023-02-12,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Wheat,Local Market,Sack,2023-02-13,Normal,Normal,Dry,Stones,Dusty,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Milk,Farmer,Container,2023-02-14,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Farmer,Sack,2023-02-15,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Turmeric,Vendor,Loose,2023-02-16,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Chilli Powder,Local Market,Loose,2023-02-17,Too Bright,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Leaves Black Residue,Sharp,Yes,Color,Moderate
Tomato,Self-grown,Open,2023-02-18,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Milk,Local Market,Container,2023-02-19,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Rice,Local Market,Sack,2023-02-20,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Wheat,Farmer,Sack,2023-02-21,Normal,Normal,Dry,Husks,Dusty,Mixed,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Turmeric,Farmer,Covered,2023-02-22,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Milk,Vendor,Container,2023-02-23,Yellowish,Chemical,Smooth,None,Layered,Mixed,Forms Layer,Excessive Foam,Plastic Smell,Bitter,Yes,Detergent,Toxic
Chilli Powder,Vendor,Loose,2023-02-24,Normal,Strong,Powdery,Seeds,Dusty,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,Yes,Stone,Low Risk
Rice,Self-grown,Container,2023-02-25,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Farmer,Container,2023-02-26,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Tomato,Vendor,Open,2023-02-27,Mixed Shades,Rotten,Sticky,None,Clumpy,Sinks,Leaves Residue,No Foam,Burns Cleanly,Not Tested,Yes,Color,Moderate
Turmeric,Local Market,Covered,2023-02-28,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Wheat,Local Market,Sack,2023-03-01,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Vendor,Container,2023-03-02,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Farmer,Sack,2023-03-03,Faded,Rotten,Wet,Insects,Clumpy,Floats,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Urea,Moderate
Chilli Powder,Local Market,Loose,2023-03-04,Normal,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,No,None,Safe
Milk,Local Market,Container,2023-03-05,Yellowish,Chemical,Sticky,None,Layered,Mixed,Forms Layer,Excessive Foam,Melts,Bitter,Yes,Detergent,Toxic
Turmeric,Vendor,Loose,2023-03-06,Too Bright,Chemical,Powdery,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Wheat,Farmer,Sack,2023-03-07,Normal,Normal,Dry,Stones,Dusty,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Tomato,Self-grown,Open,2023-03-08,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Milk,Farmer,Container,2023-03-09,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Rice,Local Market,Sack,2023-03-10,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Turmeric,Local Market,Covered,2023-03-11,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Chilli Powder,Vendor,Loose,2023-03-12,Too Bright,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Leaves Black Residue,Sharp,Yes,Color,Moderate
Milk,Vendor,Container,2023-03-13,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Wheat,Local Market,Sack,2023-03-14,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Rice,Farmer,Sack,2023-03-15,Normal,Normal,Dry,Husks,Dusty,Mixed,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Milk,Local Market,Container,2023-03-16,Yellowish,Chemical,Smooth,None,Layered,Mixed,Forms Layer,Excessive Foam,Plastic Smell,Bitter,Yes,Detergent,Toxic
Tomato,Vendor,Open,2023-03-17,Mixed Shades,Rotten,Sticky,None,Clumpy,Sinks,Leaves Residue,No Foam,Burns Cleanly,Not Tested,Yes,Color,Moderate
Turmeric,Farmer,Covered,2023-03-18,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Chilli Powder,Self-grown,Container,2023-03-19,Normal,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,No,None,Safe
Milk,Farmer,Container,2023-03-20,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Self-grown,Container,2023-03-21,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Wheat,Farmer,Sack,2023-03-22,Normal,No Smell,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Local Market,Container,2023-03-23,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Turmeric,Vendor,Loose,2023-03-24,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Rice,Vendor,Sack,2023-03-25,Faded,Rotten,Wet,Insects,Clumpy,Floats,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Urea,Moderate
Chilli Powder,Local Market,Loose,2023-03-26,Normal,Strong,Powdery,Seeds,Dusty,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,Yes,Stone,Low Risk
Tomato,Farmer,Open,2023-03-27,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Milk,Vendor,Container,2023-03-28,Yellowish,Chemical,Sticky,None,Layered,Mixed,Forms Layer,Excessive Foam,Melts,Bitter,Yes,Detergent,Toxic
Turmeric,Local Market,Covered,2023-03-29,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Wheat,Local Market,Sack,2023-03-30,Normal,Normal,Dry,Stones,Dusty,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Milk,Farmer,Container,2023-03-31,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Farmer,Sack,2023-04-01,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Turmeric,Vendor,Loose,2023-04-02,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Chilli Powder,Local Market,Loose,2023-04-03,Too Bright,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Leaves Black Residue,Sharp,Yes,Color,Moderate
Tomato,Self-grown,Open,2023-04-04,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Sweet,No,None,Safe
Milk,Local Market,Container,2023-04-05,Normal,Normal,Smooth,None,Uniform,Sinks,Leaves Residue,Slight Foam,Burns Cleanly,Sweet,Yes,Water,Low Risk
Rice,Local Market,Sack,2023-04-06,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Wheat,Farmer,Sack,2023-04-07,Normal,Normal,Dry,Husks,Dusty,Mixed,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Stone,Low Risk
Turmeric,Farmer,Covered,2023-04-08,Too Bright,Chemical,Gritty,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
Milk,Vendor,Container,2023-04-09,Yellowish,Chemical,Smooth,None,Layered,Mixed,Forms Layer,Excessive Foam,Plastic Smell,Bitter,Yes,Detergent,Toxic
Chilli Powder,Vendor,Loose,2023-04-10,Normal,Strong,Powdery,Seeds,Dusty,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,Yes,Stone,Low Risk
Rice,Self-grown,Container,2023-04-11,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Farmer,Container,2023-04-12,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Tomato,Vendor,Open,2023-04-13,Mixed Shades,Rotten,Sticky,None,Clumpy,Sinks,Leaves Residue,No Foam,Burns Cleanly,Not Tested,Yes,Color,Moderate
Turmeric,Local Market,Covered,2023-04-14,Normal,Normal,Powdery,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bitter,No,None,Safe
Wheat,Local Market,Sack,2023-04-15,Normal,Normal,Dry,None,Uniform,Sinks,Leaves Residue,No Foam,Burns Cleanly,Bland,No,None,Safe
Milk,Vendor,Container,2023-04-16,Normal,Normal,Smooth,None,Uniform,Sinks,Fully Dissolves,No Foam,Burns Cleanly,Sweet,No,None,Safe
Rice,Farmer,Sack,2023-04-17,Faded,Rotten,Wet,Insects,Clumpy,Floats,Leaves Residue,No Foam,Burns Cleanly,Bland,Yes,Urea,Moderate
Chilli Powder,Local Market,Loose,2023-04-18,Normal,Strong,Powdery,None,Uniform,Floats,Leaves Residue,No Foam,Burns Cleanly,Sharp,No,None,Safe
Milk,Local Market,Container,2023-04-19,Yellowish,Chemical,Sticky,None,Layered,Mixed,Forms Layer,Excessive Foam,Melts,Bitter,Yes,Detergent,Toxic
Turmeric,Vendor,Loose,2023-04-20,Too Bright,Chemical,Powdery,None,Uniform,Sinks,Leaves Residue,Slight Foam,Leaves Black Residue,Bitter,Yes,Lead Chromate,Toxic
"""
# Note: The '...' indicates the rest of the 1000 rows should be here. 
# For a working example, we use the provided subset.
df = pd.read_csv(io.StringIO(csv_data))

# --- 2. Preprocessing and Encoding ---

# Drop columns that are not features for the model
# Date is too specific, Suspected_Adulterant is a label not a feature for this task.
df_features = df.drop(columns=['Date_of_Collection', 'Suspected_Adulterant'])

# Define features (X) and targets (y)
X = df_features.drop(columns=['Is_Adulterated', 'Health_Risk_Level'])
y_adulterated = df_features['Is_Adulterated']
y_risk = df_features['Health_Risk_Level']

# Encode categorical features using one-hot encoding
X_encoded = pd.get_dummies(X, drop_first=True)

# Encode target labels
le_adulterated = LabelEncoder()
y_adulterated_encoded = le_adulterated.fit_transform(y_adulterated)

le_risk = LabelEncoder()
y_risk_encoded = le_risk.fit_transform(y_risk)

# --- 3. Split Data for Both Tasks ---

# A) Split for 'Is_Adulterated' prediction
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(
    X_encoded, y_adulterated_encoded, test_size=0.2, random_state=42, stratify=y_adulterated_encoded
)

# B) Split for 'Health_Risk_Level' prediction
X_train_risk, X_test_risk, y_train_risk, y_test_risk = train_test_split(
    X_encoded, y_risk_encoded, test_size=0.2, random_state=42, stratify=y_risk_encoded
)

print("Data successfully preprocessed and split.")
print(f"Training set size: {X_train_ad.shape[0]} samples")
print(f"Testing set size: {X_test_ad.shape[0]} samples")
print("-" * 50)


# --- 4. Train and Evaluate Model A: Is_Adulterated ---

print("\n--- Task A: Predicting 'Is_Adulterated' ---")
# Initialize Random Forest Classifier
# A Random Forest is a great choice as it's robust to overfitting, works well with many features,
# and can capture non-linear relationships between sensory data and adulteration.
rf_adulterated = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model
rf_adulterated.fit(X_train_ad, y_train_ad)

# Make predictions
y_pred_ad = rf_adulterated.predict(X_test_ad)

# --- 5. Evaluate and Print Classification Report for Model A ---
print("\nClassification Report for Adulteration:")
print(classification_report(y_test_ad, y_pred_ad, target_names=le_adulterated.classes_))
print(f"Accuracy: {accuracy_score(y_test_ad, y_pred_ad):.4f}")


# --- 6. Get Feature Importances for Model A ---
feature_importances_ad = pd.Series(rf_adulterated.feature_importances_, index=X_encoded.columns).sort_values(ascending=False)
print("\nTop 10 Most Important Features for Predicting Adulteration:")
print(feature_importances_ad.head(10))
print("-" * 50)


# --- 4. Train and Evaluate Model B: Health_Risk_Level ---

print("\n--- Task B: Predicting 'Health_Risk_Level' ---")
# Initialize another Random Forest Classifier for the risk level
rf_risk = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Train the model
rf_risk.fit(X_train_risk, y_train_risk)

# Make predictions
y_pred_risk = rf_risk.predict(X_test_risk)


# --- 5. Evaluate and Print Classification Report for Model B ---
print("\nClassification Report for Health Risk Level:")
print(classification_report(y_test_risk, y_pred_risk, target_names=le_risk.classes_))
print(f"Accuracy: {accuracy_score(y_test_risk, y_pred_risk):.4f}")


# --- 6. Get Feature Importances for Model B ---
feature_importances_risk = pd.Series(rf_risk.feature_importances_, index=X_encoded.columns).sort_values(ascending=False)
print("\nTop 10 Most Important Features for Predicting Health Risk:")
print(feature_importances_risk.head(10))
print("-" * 50)


# --- 7. Save the Models ---
joblib.dump(rf_adulterated, 'adulteration_model.joblib')
joblib.dump(le_adulterated, 'adulteration_label_encoder.joblib')

joblib.dump(rf_risk, 'risk_level_model.joblib')
joblib.dump(le_risk, 'risk_level_label_encoder.joblib')
joblib.dump(X_encoded.columns, 'model_features.joblib')


print("\nModels and encoders saved successfully:")
print("- adulteration_model.joblib")
print("- adulteration_label_encoder.joblib")
print("- risk_level_model.joblib")
print("- risk_level_label_encoder.joblib")
print("- model_features.joblib")

Data successfully preprocessed and split.
Training set size: 76 samples
Testing set size: 20 samples
--------------------------------------------------

--- Task A: Predicting 'Is_Adulterated' ---

Classification Report for Adulteration:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00        10
         Yes       1.00      1.00      1.00        10

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Accuracy: 1.0000

Top 10 Most Important Features for Predicting Adulteration:
Color_Normal                             0.156750
Appearance_Uniform                       0.150531
Foam_After_Mixing_Slight Foam            0.079994
Appearance_Dusty                         0.079985
Foam_After_Mixing_No Foam                0.074093
Float_or_Settle_Test_Sinks               0.072911
Smell_Normal                             0.049852
Float_o